# SETUP

In [1]:
if "google.colab" in str(get_ipython()):
    !pip install pyspark
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [2]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


zsh:1: command not found: kaggle
unzip:  cannot find or open ./data/yelp-dataset.zip, ./data/yelp-dataset.zip.zip or ./data/yelp-dataset.zip.ZIP.


In [3]:
review_filename = 'data/yelp_academic_dataset_review.json'
user_filename = 'data/yelp_academic_dataset_user.json'
business_filename = 'data/yelp_academic_dataset_business.json'

# Lavorare su un sample più piccolo (10%)
### conviente farlo solo sulle review per problemi di join

In [4]:
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext().getOrCreate()
sc = spark.sparkContext


23/12/16 14:41:04 WARN Utils: Your hostname, MacBook-Pro-di-Luca.local resolves to a loopback address: 127.0.0.1; using 192.168.1.128 instead (on interface en0)
23/12/16 14:41:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/16 14:41:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# TODO add docstring
def from_json_to_RDD(filename):
    raw_df = spark.read.json(filename)
    raw_df.show()
    #return raw_df.sample(frac=1/10, random_state = 0).dropna().rdd
    return raw_df.rdd

raw_review_RDD = from_json_to_RDD(review_filename)#.sample(True, 1, 0)  #699192 rows #shuffle
raw_user_RDD = from_json_to_RDD(user_filename)
raw_business_RDD = from_json_to_RDD(business_filename)
#print(raw_business_RDD.filter(lambda x: x[2]=='bqFG0QJY9jj2m55OqAVHeA').first())

raw_review_RDD.first()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|     name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|         

23/12/16 14:41:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1|   

Row(business_id='XQfwVwDr-v0ZS3_CbbE5Xw', cool=0, date='2018-07-07 22:09:11', funny=0, review_id='KU_O5udG6zpxOg-VcAEodg', stars=3.0, text="If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", useful=0, user_id='mh_-eMZ6K5RLWhZyISBhwA')

In [6]:
raw_review_RDD_bis = raw_review_RDD.sample(False, 1/1000, 0)
print(raw_review_RDD_bis.count())

user_set = set(raw_review_RDD_bis.map(lambda x: x[-1]).collect())
print(len(user_set))

7046


6879


In [7]:
#sample number of users and take all the reviews made by the users in the subset

raw_user_RDD = raw_user_RDD.filter(lambda x: (x[-2] in user_set))
raw_user_RDD.count()
#13608



6879

# RETRIEVE RDDs

In [8]:
import hashlib

def hash(s):
    h = hashlib.sha1(s.encode())
    return int(h.hexdigest(),16)%((2**31)-1)

In [9]:
def get_review_tuple(entry):
    """ Parse a row in the review dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (hash(str(entry["review_id"])),    # 0
            hash(str(entry["user_id"])),      # 1
            hash(str(entry["business_id"])),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.first()}')

review_RDD.filter(lambda x: x[1]==1328793112).collect()

There are 6990280 reviews in the dataset
Reviews: (828134966, 1383820667, 2072010094, 3, 0, 0, 0, "If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.")


[(754040663,
  1328793112,
  1322585576,
  5,
  0,
  0,
  1,
  'Remarkable food with beach access for the whole family, flip flops acceptable.  You step up to the counter, order, get a number and bus you own food to a table inside or out.  The locals know about this place and really love it just as we did on our first visit.  Highly recommended.  Probably good enough to drive from Santa Barbara to eat here.  Local beer on tap, kids are more than welcome.'),
 (1467518570,
  1328793112,
  1034846065,
  5,
  30,
  7,
  32,
  'I loved everything about this lovely train station.  The staff was friendly and helpful.  The station was beautiful.  Every aspect of passing through was wonderful.  It is a great place.  Thank you Philadelphia for making our vacation special and providing such a great place.'),
 (1378804539,
  1328793112,
  1297517353,
  4,
  2,
  1,
  2,
  'The classic facade, built over an earlier facing, was copied from a first century Roman book, Spanish edition, published in Ma

In [10]:
def get_user_tuple(entry):
    """ Parse a row in the user dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the user dataset in JSON format
    Returns:
        tuple: (user_id, name, review_count, average_stars, useful, funny, cool, fans)
    """

    return (hash(str(entry["user_id"])),          # 0
            str(entry["name"]),             # 1
            int(entry["review_count"]),     # 2
            float(entry["average_stars"]),  # 3
            int(entry["useful"]),           # 4
            int(entry["funny"]),            # 5
            int(entry["cool"]),             # 6
            int(entry["fans"]))             # 7


user_RDD = raw_user_RDD.map(get_user_tuple)

user_count = user_RDD.count()

print(f'There are {user_count} users in the dataset')
print(f'Users: {user_RDD.first()}')


There are 6879 users in the dataset
Users: (67330113, 'Jelena', 1807, 4.06, 17331, 10461, 13734, 828)


In [11]:
def get_business_tuple(entry):
    """ Parse a row in the business dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the business dataset in JSON format
    Returns:
        tuple: (business_id, name, city, state, stars, review_count, categories)
    """

    categories = [] if entry["categories"] == None \
                    else str(entry["categories"]).split(", ")
    
    return (hash(str(entry["business_id"])),  # 0
            str(entry["name"]),         # 1
            str(entry["city"]),         # 2
            str(entry["state"]),        # 3
            float(entry["stars"]),      # 4
            int(entry["review_count"]), # 5
            categories)                 # 6

#TODO Attributes?

business_RDD = raw_business_RDD.map(get_business_tuple)

business_count = business_RDD.count()
# print(business_RDD.filter(lambda x: x[0]=='1H8ReY5GlGcHJz7umVidkg').first())

print(f'There are {business_count} business in the dataset')
print(f'Business: {business_RDD.first()}')

There are 150346 business in the dataset
Business: (1768608060, 'Abby Rappoport, LAC, CMQ', 'Santa Barbara', 'CA', 5.0, 7, ['Doctors', 'Traditional Chinese Medicine', 'Naturopathic/Holistic', 'Acupuncture', 'Health & Medical', 'Nutritionists'])


# Calculate overall
### - for each business_id:

$$
  \Delta = \dfrac{1}{2} 
              \left( 
                  \dfrac{\text{useful} + \dfrac{1}{2}(\text{funny} + \text{cool})} 
                        {\text{best\ useful} + \dfrac{1}{2}(\text{best\ funny} + \text{best\ cool})}
                        + 
                  \dfrac{\text{fans}}
                        {\text{best\ fans}}
              \right)
$$
$$
\Delta : [0, 1]
$$
$$
  \text{overall} = \begin{cases} 
              \text{stars} + \Delta & \text{if stars } \ge 3\\ 
              \text{stars} - \Delta & \text{if stars } \lt 3
            \end{cases}
$$

In [12]:
def get_business_id_useful_funny_cool(row):
    """
        Writes a tuple with key business_id and attributes useful, funny, cool
    """
    return (row[2], (row[4], row[5], row[6]))

def get_max_useful_funny_cool(row1, row2):
    """
        Returns a new RDD with, for each business_id key, the maxima values of useful, funny, cool
    """
    return tuple(max(row1[i], row2[i]) for i in range(3))

review_tuple_RDD = review_RDD.map(get_business_id_useful_funny_cool)
review_best_ufc_RDD = review_tuple_RDD.reduceByKey(get_max_useful_funny_cool)

review_best_ufc_RDD.take(5)

# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 


[(94502400, (40, 33, 37)),
 (1749679040, (11, 3, 7)),
 (1992874600, (25, 8, 16)),
 (898716160, (8, 6, 9)),
 (21494360, (11, 7, 10))]

In [13]:
def rearrange_review_and_best_ufc_RDD(row):
    """
        Merge, for each business_id, the row values with the best useful, funny, cool ones
    """
    return (row[2], (row[0], row[1], row[3], row[4], row[5], row[6]))

review_ufc_RDD = review_RDD.map(rearrange_review_and_best_ufc_RDD).join(review_best_ufc_RDD)
review_ufc_RDD.first()

# (id_business, ((id_review, id_user, star, useful, funny , cool), (best useful, best funny, best cool)))


(94502400, ((57478457, 1874891613, 5, 2, 5, 2), (40, 33, 37)))

In [14]:
def partial_overall(x):
    """
        Rearrange review_ufc_RDD and calculates the partial overall of the delta above
        partial_overall ={[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2]
    """
    key = (x[1][0][0], x[1][0][1], x[0], x[1][0][2])
    num = (x[1][0][3] + (x[1][0][4] + x[1][0][5])/2)
    den = (x[1][1][0] + (x[1][1][1] + x[1][1][2])/2)
    return (*key,(num/den if sum(x[1][1]) != 0 else 0))


partial_review_overall_RDD = review_ufc_RDD.map(partial_overall)
partial_review_overall_RDD.first()

# tuple: (review_id, user_id, buisness_id, stars, partial overall) 

(57478457, 1874891613, 94502400, 5, 0.07333333333333333)

In [15]:
# tuple: (user_id, fans)
user_fans_RDD = user_RDD.map(lambda x: (x[0], x[7]))

# tuple: (user_id, (review_id, buisness_id, stars, partial overall), fans)
partial_overall_fans_RDD = partial_review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3], x[4]))).join(user_fans_RDD)

partial_overall_fans_RDD.first()


(2133117606, ((1955486772, 1246376960, 4, 0.0), 17))

In [16]:
# user with the highest number of fans
best_user_fans = user_RDD.max(lambda x: x[7])
print("User with the highest number of fans: ",best_user_fans)


def review_overall(x):
    user_id, review_id, business_id, stars= x[0], x[1][0][0], x[1][0][1], x[1][0][2]
    value = stars + (x[1][0][3]+x[1][1]/best_user_fans[7])/(2 if stars >=3 else -2) 
    return (review_id, user_id, business_id, value)

    #tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))


review_overall_RDD = partial_overall_fans_RDD.map(review_overall)

review_overall_RDD.sortBy(lambda x:x[3], ascending=False).take(5)


User with the highest number of fans:  (652197590, 'Fox', 17473, 3.77, 206296, 185823, 195814, 3493)


[(1094388126, 652197590, 861841120, 6.0),
 (1259756670, 652197590, 10838641, 6.0),
 (1228519224, 652197590, 55963282, 6.0),
 (1884331732, 652197590, 2074152642, 6.0),
 (1744371905, 652197590, 391408243, 6.0)]

# Basic Recommendation

In [17]:
#Keeping businesses that have at least 10 reviews
threshold = 10
review_by_business_filtered_RDD = review_overall_RDD.map(lambda x: (x[2], (x[0], x[1], x[3]))).groupByKey()\
                                                    .mapValues(list).filter(lambda x: len(x[1])>=threshold)
review_by_business_filtered_RDD.take(5)


[(1071342742,
  [(2086080823, 2133117606, 3.017584953456697),
   (1974841334, 1405334122, 4.015151515151516),
   (1328740603, 1014261759, 5.061608064614076),
   (125832719, 1279246925, 0.9842759111296185),
   (1582433172, 1279246925, 0.9994274262811337),
   (704749741, 1548714086, 3.0305893171624634),
   (473234367, 1173822726, 1.9695538262672532),
   (2089214103, 1878144096, 4.038737648457087),
   (1069582819, 197356793, 4.0001431434297166),
   (891213156, 197356793, 4.0001431434297166),
   (178370397, 870693175, 4.0),
   (345911816, 1624531909, 5.034597333194528),
   (1575118385, 215897804, 4.0030060120240485),
   (1493459276, 215897804, 3.010581769599806),
   (403428973, 122187143, 5.0),
   (756900574, 1776167579, 3.0177280968864135),
   (852800, 423423035, 5.076505825503822),
   (332569539, 155755016, 3.0),
   (1540815865, 1120573805, 4.175101284820724)]),
 (2006352312,
  [(1169647652, 483050162, 4.0030060120240485),
   (2112275129, 2038464272, 4.035477882171269),
   (71009950, 762

In [18]:
def sortFunction(tuple):
    """ Construct the sort string (does not perform actual sorting)
    Args:
        tuple: (rating, MovieName)
    Returns:
        sortString: the value to sort with, 'rating MovieName'
    """

    if (tuple[1][1]!= None):
        value = '{:.3f}'.format(tuple[1][1])
    else:
        value = ''
    key = tuple[0]
    return (value + ' ' + str(key))


In [19]:
#Make, for each business, the average of the stars for all the reviews

business_overall_RDD = review_by_business_filtered_RDD.map(lambda x: (x[0],  sum(i[2] for i in x[1])/len(x[1])))
business_overall_RDD.first()

(1071342742, 3.549196593232033)

In [20]:
# top recommended businesses
business_RDD.map(lambda x: (x[0], (x[1], x[6]))).leftOuterJoin(business_overall_RDD).sortBy(sortFunction, False).take(10)  #x[2], x[3],.filter(lambda x: 'Restaurants' in x[1][1])


[(997631831,
  (('Coffee N’ Comics',
    ['Books',
     'Mags',
     'Music & Video',
     'Breakfast & Brunch',
     'Shopping',
     'Restaurants',
     'Sandwiches',
     'Comic Books',
     'Food',
     'Coffee & Tea']),
   5.293831903438212)),
 (1456570901,
  (('Maverik', ['Automotive', 'Gas Stations']), 5.236978315285615)),
 (65028072,
  (('BeiJing House', ['Asian Fusion', 'Chinese', 'Restaurants', 'Szechuan']),
   5.209125562331974)),
 (650326991,
  (('Samson and Sons Heating & Cooling',
    ['Home Services', 'Heating & Air Conditioning/HVAC']),
   5.198813731358822)),
 (1568354545,
  (('WWOZ FM Radio', ['Radio Stations', 'Mass Media']), 5.187162643739861)),
 (32917614,
  (('Paleteria Del Angel',
    ['Specialty Food', 'Ice Cream & Frozen Yogurt', 'Desserts', 'Food']),
   5.175840052197418)),
 (1488295571,
  (('8th Day Distillery',
    ['Venues & Event Spaces',
     'Event Planning & Services',
     'Food',
     'Bars',
     'Nightlife',
     'Distilleries']),
   5.1665980043711

# Collaborative Filtering

In [21]:
review_overall_bis = review_overall_RDD.map(lambda x: (x[1], x[2], x[3]))


In [22]:
#Cosine Distance
def cosine_dist(value):
    """ 
    arg: utente da confrontare
    per ogni utente:
        - user_id, prodotto tra rating di item per stessa chiave di item, lista di rating
        - somma di valori nel numeratore, somma di liste del denominatore
        - prodotto delle somma quadratiche nel denominatore tra i due utenti
        - prendi utenti che hanno qualche voto su un business in comune diverso da zero
    """
    value_review = dict(review_overall_bis.filter(lambda x:x[0] == value).map(lambda x:(x[1],x[2])).collect()) #business: voto
    print(value_review)
    if (len(value_review)==0): 
        return None, None
    user_sim = review_overall_bis.map(lambda x: (x[0], (0 if x[1] not in value_review.keys() \
                                                        else x[2]*value_review[x[1]], [x[2]])))\
                                 .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
                                 .map(lambda x:(x[0], x[1][0]/((sum(i**2 for i in x[1][1])**0.5) * \
                                                               (sum(i**2 for i in value_review.values())**0.5))))\
                                 .filter(lambda x:x[1]!=0 and x[0]!=value).take(5) #prendiamo i 5 utenti più simili
    # TODO nel repo mettere sviluppi futuri, come prendere tutta la lista di user_sim e confrontare l'RMSE con UV decomprosition
    return sorted(user_sim), value_review

def collaborative_filtering(value):
    #print(value)
    user_sim, value_review = cosine_dist(value)
    if (user_sim == None and value_review == None):
        print("Unable to do the Collab Filtering for this user")
        return
    #print(value_review.keys())
    return review_overall_bis.filter(lambda x: (x[0] in dict(user_sim).keys() and x[1] not in value_review.keys()))\
                             .take(10) #.sortBy(lambda x:x[2],ascending = False)


In [23]:
# print(review_overall_bis.take(10)[5])
business_collab = collaborative_filtering(652197590) # 79992688
# print(review_overall_RDD.filter(lambda x:x[1]=='Eep1pCr1zMqub16tYgUFgQ').collect())
business_collab
#TODO make print of the business as name, categories...

{861841120: 6.0, 1689272400: 3.7, 560629680: 3.888888888888889, 1997104721: 4.55, 10838641: 6.0, 2033874961: 4.928571428571429, 879801121: 3.8095238095238093, 1833563441: 5.0, 288797201: 0.0, 55963282: 6.0, 2074152642: 6.0, 1590266082: 5.0, 838883202: 4.5911330049261085, 1262233522: 4.848484848484849, 922981442: 4.0, 391408243: 6.0, 1079872403: 4.666666666666667, 1404188483: 4.52930056710775, 1401786404: 1.3333333333333335, 54304324: 3.916666666666667, 780786164: 4.6048780487804875, 1768611845: 5.958823529411765, 416920005: 3.6721311475409837, 1966367685: 3.674074074074074, 588937046: 3.8636363636363638, 2146418406: 4.948717948717949, 1072960246: 6.0, 783522247: 5.723618090452261, 574007767: 1.0, 813149368: 0.0, 2135222328: 3.8235294117647056, 537283928: 4.944444444444445, 1746329448: 4.616666666666667, 305339448: 4.893939393939394, 560272088: 4.0, 1221740808: 6.0, 832761368: 5.787313432835821, 411055688: 5.931818181818182, 703578169: 5.587962962962963, 1888035690: 5.0, 1845120250: 4.0

[(483050162, 1688587680, 4.1905060120240485),
 (483050162, 958996320, 5.0030060120240485),
 (483050162, 440353520, 4.0655060120240485),
 (483050162, 461041840, 3.0982441072621434),
 (483050162, 267142480, 5.0352640765401775),
 (483050162, 1989312240, 4.0030060120240485),
 (483050162, 690127920, 5.160900748866153),
 (483050162, 1868396001, 5.03788973295428),
 (483050162, 470109921, 5.0030060120240485),
 (483050162, 356161761, 3.069672678690715)]

In [24]:
# recommended_business = sorted(business_collab, key=lambda x : -x[2])
business_RDD.filter(lambda x: x[0] in [i[1] for i in business_collab]).collect()

[(1688587680,
  'Last Chance Thrift Store',
  'Palm Harbor',
  'FL',
  3.5,
  34,
  ['Thrift Stores', 'Shopping']),
 (1868396001,
  'Golden Dinosaurs',
  'Gulfport',
  'FL',
  4.5,
  158,
  ['Coffee & Tea',
   'Pubs',
   'Delis',
   'Food',
   'Vegan',
   'Cuban',
   'Restaurants',
   'Bars',
   'Nightlife']),
 (958996320,
  'Cedars Restaurant',
  'Seminole',
  'FL',
  4.5,
  73,
  ['Halal', 'Restaurants', 'Lebanese', 'Middle Eastern']),
 (440353520,
  "Topper's Creamery",
  'Largo',
  'FL',
  4.0,
  35,
  ['Food', 'Ice Cream & Frozen Yogurt']),
 (461041840,
  'Bollywood Aroma Indian Cuisine',
  'Kenneth City',
  'FL',
  4.0,
  61,
  ['Halal', 'Indian', 'Restaurants']),
 (470109921,
  'The Barn at Starkey Market',
  'New Port Richey',
  'FL',
  4.5,
  14,
  ['Bars', 'Nightlife']),
 (356161761,
  "Jimmy John's",
  'St. Petersburg',
  'FL',
  2.5,
  33,
  ['Fast Food',
   'Food Delivery Services',
   'Sandwiches',
   'Delis',
   'Food',
   'Restaurants']),
 (267142480,
  'Nebraska Mini M

# UV Decompostion

## Initialization

In [25]:
#We need to train and predict the vectors U and V
# train(trainingRDD, rank, iterations, regularization_parameter)  --> (U,V)
#We need to calculate RMSE between the utility and our matrix

UV_rank = 2
# regularization_parameter = 0.1

U_RDD = user_RDD.map(lambda x: (x[0], [1 for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [1 for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])

# U_RDD.take(5)
# V_RDD.take(5)


In [26]:
review_overall_bis = review_overall_RDD.map(lambda x: (x[1], x[2], x[3]))
#review_overall_bis = review_RDD.map(lambda x: (x[1], x[2], x[3]))


In [27]:
#NOTE We are bypassing at the moment the validation set (useful for achieving the best rank)
trainingRDD, testRDD = review_overall_bis.randomSplit([7,3], seed=0)

In [28]:
M_BC = sc.broadcast(trainingRDD.collect())


In [29]:
V_BC = sc.broadcast(dict(V_RDD.collect()))


In [30]:
U_BC = sc.broadcast(dict(U_RDD.collect()))


## Performing Optimization

In [31]:
# normalized-->tuple: (user_id, buisness_id, overall - normalized_value)
# review_overall --> tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))

def update_U(entry):
    id = entry[0]
    #print("id=",id)
    U_value = entry[1]
    reviews = [i for i in M_BC.value if i[0] == id]
    for s in range(UV_rank):
        #print(UV_rank)
        res = 0
        den = 0
        #print("reviews= ",reviews)

        for review in reviews:
            j = review[1] # business_id
            p = 0
            for k in [i for i in range(UV_rank) if i != s]:
                #print(V_BC.value)
                
                p += U_value[k] * V_BC.value[j][k]
                #print("p=",p)
            res += V_BC.value[j][s] * (review[2] - p)
            den += V_BC.value[j][s]**2
        
        U_value[s] = res / (den if den > 0 else 1)\
            #if len(reviews)>0 else 1

    return (id, U_value)


In [32]:
def update_V(entry):
    id = entry[0]
    #print("id=",id)
    V_value = entry[1]
    reviews = [i for i in M_BC.value if i[1] == id]

    # out = [1]*rank
    for s in range(UV_rank):
        res = 0
        den = 0
        #print("reviews= ",reviews)
        for review in reviews:
            j = review[0] # user_id
            p = 0
            for k in [i for i in range(UV_rank) if i != s]:
                p += V_value[k] * U_BC.value[j][k]
                #print("p=",p)
            res += U_BC.value[j][s] * (review[2] - p)
            den += U_BC.value[j][s]**2
        
        V_value[s] = (res / (den if den > 0 else 1))\
#            if len(reviews)>0 else 1

    return (id, V_value)


# V_RDD.map(lambda x :  update_V(x)).take(20)

In [33]:
def train(U_RDD,V_RDD,iterations):  
# iterations = 2
    global V_BC, U_BC
    for i in range(iterations):
        V_BC = sc.broadcast(dict(V_RDD.collect()))
        U_RDD = U_RDD.map(update_U)

        U_BC = sc.broadcast(dict(U_RDD.collect()))
        V_RDD = V_RDD.map(update_V)

    return U_RDD.collect(), V_RDD.collect()
    

In [34]:
U,V = train(U_RDD, V_RDD, iterations =2)

In [35]:
U

[(67330113, [3.511088385495061, 0.9738132576168349]),
 (1297302066, [2.5742878856073417, 0.9433700630601284]),
 (760041815, [3.7295532605552424, 0.8396909406592404]),
 (41225982, [3.3990946102657156, 0.9278767041622437]),
 (55754239, [3.598627479965088, 0.9579972753153692]),
 (1732647300, [2.7093772554171967, 1.0224627984344818]),
 (850949409, [2.758905984695952, 1.0055507890969184]),
 (401029605, [3.315466432292394, 0.9435623675372647]),
 (1157496782, [1.9863335538800497, 0.9973252128407148]),
 (155028714, [2.8984669750577456, 0.9400180227322482]),
 (543719164, [3.001456432045286, 0.9962172682975762]),
 (1879776525, [2.7566288011314173, 0.932988775420198]),
 (312637553, [3.0317767881936404, 1.0001189611733572]),
 (1217174437, [2.758099419546976, 1.027348904983974]),
 (1127570257, [2.956562483405954, 1.074694309822327]),
 (1571642927, [2.5737802118839554, 0.9837688173957219]),
 (399149528, [2.7160044780328376, 1.0103752548713558]),
 (472228578, [2.5812149060883707, 1.045170183565275]),

In [36]:
V

[(1768608060, [0.0, 0.0]),
 (674023133, [0.0, 0.0]),
 (1853240994, [0.906148254068146, 1.134983883971752]),
 (25885286, [0.8156889685571377, 1.2779383192926246]),
 (886402536, [1.2448612561382733, 1.0]),
 (899415875, [1.2472950638075864, 1.0]),
 (151330391, [0.0, 0.0]),
 (659900618, [0.0, 0.0]),
 (2073408338, [0.0, 0.0]),
 (259476066, [0.0, 0.0]),
 (1270265292, [0.9669037812531486, 1.0277558418043546]),
 (901865183, [1.4963907661577918, 1.0000000000000009]),
 (399025230, [0.0, 0.0]),
 (1188362622, [0.0, 0.0]),
 (1652935626, [1.084596899568361, 1.2676059921752514]),
 (905182201, [1.0976571281117082, 1.060499652102892]),
 (464645059, [0.0, 0.0]),
 (1007062082, [0.0, 0.0]),
 (1721829350, [0.0, 0.0]),
 (1979569868, [1.1231533301137118, 0.8133043368065834]),
 (464040412, [1.0329623234779726, 0.6017136696000452]),
 (904870512, [1.0313709967375628, 1.058301738795664]),
 (464545888, [0.8005868320871692, 1.0]),
 (358680582, [0.7305075286105822, 0.9045942058279821]),
 (395093298, [0.0, 0.0]),
 (

In [37]:
#Take an user-id and suggest business that he has not reviewed

def predict(u_value, business_value):
    return sum(u_value[i]* business_value[i] for i in range(len(u_value)))


def print_prediction(username, business_name):

    U_dict = dict(U)
    V_dict = dict(V)
    un = user_RDD.filter(lambda x: x[0]== username).first()[1]
    bn = business_RDD.filter(lambda x: x[0]== business_name).first()[2]
    print("User:",un, "----- Business:",bn, ":",predict(U_dict[username], V_dict[business_name]))



In [38]:
#RMSE of UV Decomposition

def UV_rmse(testRDD):
    rmse = 0
    U_dict = dict(U)
    V_dict = dict(V)
    for i in testRDD.collect():
        rmse += (i[2] - predict(U_dict[i[0]], V_dict[i[1]]))**2
    return (rmse/len(M_BC.value))**0.5

print("RMSE:", UV_rmse(testRDD))


RMSE: 1.2885288398921437


Try to populate U and V with the average value of review rating

In [39]:
"""avg_value = review_overall_bis.map(lambda x: (1, x[2])).reduceByKey(lambda x,y:(x+y)/2).first()[1]
avg_value"""

'avg_value = review_overall_bis.map(lambda x: (1, x[2])).reduceByKey(lambda x,y:(x+y)/2).first()[1]\navg_value'

In [40]:
"""U_RDD = user_RDD.map(lambda x: (x[0], [avg_value for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [avg_value for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])"""


'U_RDD = user_RDD.map(lambda x: (x[0], [avg_value for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])\nV_RDD = business_RDD.map(lambda x: (x[0], [avg_value for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])'

In [41]:
#U,V = train(U_RDD, V_RDD, iterations =2)

In [42]:
"""#RMSE of UV Decomposition
UV_loss = UV_rmse(testRDD)
print("RMSE:", UV_loss)"""


'#RMSE of UV Decomposition\nUV_loss = UV_rmse(testRDD)\nprint("RMSE:", UV_loss)'

Compare UV Model with ALS model (using the implementation of movie_recommender.ipynb)

In [43]:
trainingRDDALS, validationRDDALS, testRDDALS = review_overall_bis.randomSplit([6, 2, 2], seed=0)

print('Training: %s, validation: %s, test: %s\n' % (trainingRDDALS.count(),
                                                    validationRDDALS.count(),
                                                    testRDDALS.count()))
print(trainingRDDALS.take(3))
print(validationRDDALS.take(3))
print(testRDDALS.take(3))

Training: 177364, validation: 59402, test: 59737

[(2133117606, 1246376960, 4.002433438305181), (2133117606, 1802972320, 5.002433438305181), (2133117606, 2062742881, 4.105881714167251)]
[(2133117606, 804763840, 4.096183438305181), (2133117606, 884670321, 4.0357667716385155), (2133117606, 1956675204, 4.057988993860738)]
[(2133117606, 1301522241, 3.1076965961999186), (2133117606, 857422242, 3.007752587241352), (2133117606, 857422242, 5.0104121617094375)]


In [44]:
import math

def computeError(predictedRDD, actualRDD):
    """ Compute the root mean squared error between predicted and actual
    Args:
        predictedRDD: predicted ratings for each movie and each user where each entry is in the form
                      (UserID, BusinessID, Rating)
        actualRDD: actual ratings where each entry is in the form (UserID, BusinessID, Rating)
    Returns:
        RSME (float): computed RSME value
    """
    # Transform predictedRDD into the tuples of the form ((UserID, BusinessID), Rating)
    predictedReformattedRDD = predictedRDD.map(lambda i: ((i[0], i[1]), i[2]))

    # Transform actualRDD into the tuples of the form ((UserID, BusinessID), Rating)
    actualReformattedRDD = actualRDD.map(lambda i: ((i[0], i[1]), i[2]))

    # Compute the squared error for each matching entry (i.e., the same (User ID, Business ID) in each
    # RDD) in the reformatted RDDs using RDD transformtions - do not use collect()
    squaredErrorsRDD = (predictedReformattedRDD
                        .join(actualReformattedRDD)
                        .map(lambda i: math.pow(i[1][0] - i[1][1], 2))
                       )

    # Compute the total squared error - do not use collect()
    totalError = squaredErrorsRDD.reduce(lambda a, b: a+b)

    # Count the number of entries for which you computed the total squared error
    numRatings = squaredErrorsRDD.count()

    # Using the total squared error and the number of entries, compute the RSME
    return math.pow(float(totalError) / numRatings, 0.5)

In [45]:
from pyspark.mllib.recommendation import ALS

validationForPredictRDD = validationRDDALS.map(lambda i: (i[0], i[1]))

iterations = 2 # 5
regularizationParameter = 0.1
ranks = [2, 4, 8, 12]
errors = [0, 0, 0, 0]
err = 0
tolerance = 0.02

minError = float('inf')
bestRank = -1
bestIteration = -1
for rank in ranks:
    model = ALS.train(trainingRDDALS, rank, iterations=iterations,
                      lambda_=regularizationParameter)
    predictedRatingsRDD = model.predictAll(validationForPredictRDD)
    error = computeError(predictedRatingsRDD, validationRDDALS)
    errors[err] = error
    err += 1
    print('For rank {} the RMSE is {}'.format(rank, error))
    if error < minError:
        minError = error
        bestRank = rank

print('The best model was trained with rank {}'.format(bestRank))

23/12/16 14:55:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/16 14:55:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/12/16 14:55:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


For rank 2 the RMSE is 4.375531983310431


For rank 4 the RMSE is 3.872403512300409


For rank 8 the RMSE is 3.1018861217279596


For rank 12 the RMSE is 2.642106577995846
The best model was trained with rank 12


In [46]:
myModel = ALS.train(trainingRDDALS, bestRank, iterations=iterations,
                    lambda_=regularizationParameter)
testForPredictingRDDALS = testRDDALS.map(lambda i: (i[0], i[1]))
predictedTestRDDALS = myModel.predictAll(testForPredictingRDDALS)

testRMSE = computeError(testRDDALS, predictedTestRDDALS)

print('The model had a RMSE on the test set of {}'.format(testRMSE))

The model had a RMSE on the test set of 2.624682580602659


## ALS vs UV vs ? collaborative filtering ?

In [48]:
model = ALS.train(trainingRDD, 2, iterations=2,
                      lambda_=regularizationParameter)
testForPredictingRDD = testRDD.map(lambda i: (i[0], i[1]))
predictedRatingsRDD = model.predictAll(testForPredictingRDD)

print("RMSE of ALS: ", computeError(testRDD, predictedRatingsRDD))
print("RMSE of UV: ", UV_rmse(testRDD))

RMSE of ALS:  4.322198251739923


RMSE of UV:  1.2885288398921437
